# Nombre: Arturo Lazcano
## RUT: 20.470.051-6

# CC3001 Primavera 2020 Tarea 2

## Shellsort

### Profesores
Sección 1 Benjamín Bustos •
Sección 2 Jérémy Barbay •
Sección 3 Patricio Poblete / Nelson Baloian


El objetivo de esta tarea es que usted implemente el algoritmo de ordenación Shellsort.

Para describir cómo funciona Shellsort definamos una "$d$-tajada" de un arreglo como una subsecuencia de sus elementos tal que que cada uno de ellos está a $d$ casilleros de distancia del siguiente. Por ejemplo, en el siguiente diagrama se muestran en distintos colores las posible $3$-tajadas de un arreglo dado:

![shellsort1](https://github.com/ppoblete/CC3001-2020-2-Tareas/blob/master/shellsort1.png?raw=1)

Noten que no todas las $d$-tajadas tienen necesariamente el mismo número de casilleros, y observen también que una "$1$-tajada" sería el arreglo completo.

Una pasada de Shellsort consiste en elegir un valor de $d$ y luego aplicar ordenación por inserción a cada $d$-tajada por separado. El arreglo resultante se dice que está "$d$-ordenado". Por ejemplo, la siguiente figura muestra el arreglo anterior una vez que ha sido $3$-ordenado:

![shellsort2](https://github.com/ppoblete/CC3001-2020-2-Tareas/blob/master/shellsort2.png?raw=1)

Para ordenar el arreglo completo, Shellsort hace una secuencia de pasadas, con un conjunto decreciente de valores $d_k,d_{k-1}, \ldots,d_1$, con $d_1=1$. Esto último asegura que el arreglo quede finalmente ordenado, pero las pasadas anteriores contribuyen a acelerar el proceso, porque hay un teorema (que no les pedimos demostrar) que dice que si un arreglo que ya estaba $i$-ordenado se $j$-ordena, el arreglo resultante sigue estando $i$-ordenado. Esto es, una pasada no echa a perder lo que han hecho las anteriores.

# Recuerdo de la ordenación por inserción

Recuerde que la ordenación por inserción está implementada en el apunte de la manera siguiente:

In [140]:
def ordena_insercion(a):
    """Ordena el arreglo a por inserción"""
    n=len(a)
    for k in range(0,n):
        insertar(a,k)
        
def insertar(a, k):
    """
    Inserta a[k] entre los elementos anteriores
    preservando el orden ascendente (versión 2)
    """
    b=a[k] # b almacena transitoriamente al elemento a[k]
    j=k # señala la posición del lugar "vacío"
    while j>0 and b<a[j-1]:
        a[j]=a[j-1]
        j-=1
    a[j]=b

Probemos esto para asegurarnos que funcione bien:

In [141]:
import numpy as np

def verifica_ordenado(a):
    for i in range(0,len(a)-1):
        assert a[i]<=a[i+1]
    print("Arreglo ordenado OK.")
        
A = np.array([46,35,95,21,82,70,72,56,64,50])
ordena_insercion(A)
print(A)
verifica_ordenado(A)

[21 35 46 50 56 64 70 72 82 95]
Arreglo ordenado OK.


---
# Lo que usted tiene que hacer:

## 1) Programar la ordenación de una $d$-tajada

Modifique el código anterior para que en lugar de ordenar el arreglo completo, ordene solo la $d$-tajada que comienza en el casillero $i$:

In [142]:
def ordena_tajada_insercion(a,i,d):
    """Ordena la d-tajada que comienza en a[i] por inserción."""
    # escriba aquí el código modificado
    n=len(a)
    # Llamaremos a una funcion similar a 'insertar' pero modificada a una d-tajada
    for k in range(i,n,d):
        insertar_d(a,k,d)
        
# Misma funcion que la función insertar pero saltándose d espacios
def insertar_d(a, k, d):
    b=a[k] # b almacena transitoriamente al elemento a[k]
    j=k # señala la posición del lugar "vacío"
    while j>0 and j-d>=0 and b<a[j-d]:
        a[j]=a[j-d]
        j-=d
    a[j]=b

Pruebe aquí que su algoritmo $3$-ordena correctamente los elementos amarillos:

In [143]:
A = np.array([46,35,95,21,82,70,72,56,64,50])
ordena_tajada_insercion(A,2,3)
print(A)

[46 35 64 21 82 70 72 56 95 50]


## 2) Programar una pasada de Shellsort

A continuación programe una función que haga una pasada de Shellsort, dado un arreglo $a$ y el valor de $d$. Esta función debe aplicar ``ordena_tajada_inserción`` sobre cada una de las $d$-tajadas de $a$. 

In [144]:
def pasada_Shellsort(a,d):
    """Hace una pasada de Shellsort"""
    n=len(a)
    # Hace una d-tajada donde el primer elemento va recorriendo la lista (esto no afecta
    # el orden pues siempre se mantiene en forma ascendente)
    for i in range(0,n):
        ordena_tajada_insercion(a,i,d)
    
        

Luego pruebe esto y comprueba que da el mismo resultado que el ejemplo más arriba:

In [145]:
A = np.array([46,35,95,21,82,70,72,56,64,50])
pasada_Shellsort(A,3)
print(A)

[21 35 64 46 56 70 50 82 95 72]


## 3) Programar Shellsort

Con esto ya estamos listos para programar Shellsort, haciendo una secuencia de pasadas, variando el valor de $d$ y terminando con $d=1$.Hay muchas formas conocidas de generar la secuencia de valores de $d$, con variados niveles de eficiencia. A continuación, programe Shellsort usando una secuencia decreciente de valores de la forma $d_k=2^k-1$, esto es: $\ldots, 63, 31, 15, 7, 3, 1$. Se sabe que esta secuencia hace que Shellsort funcione en tiempo $\Theta(n^{3/2})$:

In [146]:
def Shellsort(a):
    """Ordena a usando Shell Sort, con la secuencia de valores …,63,31,15,7,3,1"""
    n=len(a)
    K=aux(n)
    # Se crea una lista vacía D donde se le irán agregando valores d que son potencias de 2 menos 1
    D=[]
    while K>0:
        D.append(2**K - 1)
        K-=1
    # Para cada valor d en la lista D, se hará una d-tajada partiendo en cada elemento
    # de la lista
    for d in D:
        pasada_Shellsort(a,d)
        
# Función auxiliar que tiene como objetivo determinar un k en específico para la realización
# de Shellsort, que en este caso, k será el valor tal que 2^k - 1 sea el valor más cercano a n
def aux(n):
    if n==0:
        return 2
    else:
        for i in range(0,n):
            if 2**i - 1 >= n:
                K=i
                break
        if abs((2**K - 1) - n) <= abs((2**(K-1) - 1) - n):
            return K
        else:
            K=K-1
            return K 

Pruebe aquí su algoritmo de la manera siguiente:

In [147]:
A = np.array([46,35,95,21,82,70,72,56,64,50])
Shellsort(A)
print(A)
verifica_ordenado(A)

[21 35 46 50 56 64 70 72 82 95]
Arreglo ordenado OK.


En la siguiente celda agregue una prueba similar de ordenación de un arreglo de tamaño $1000$ generado al azar (sin imprimirlo):

In [148]:
# Se creará una lista de largo 1000 de números al azar entre 0 y 1  y se comprueba el ordenamiento
A=np.random.random(1000)
Shellsort(A)
verifica_ordenado(A)

Arreglo ordenado OK.


## 4) Probar con una secuencia diferente de valores $d_k$

Finalmente, investigue respecto de otras maneras de generar la secuencia $d_k$, escoja una secuencia en particular, modifique su versión de Shellsort que la use y pruébela.

In [149]:
# Al igual que antes, se crea una lista vacía D para ir agregando términos que se forman
# a partir de la expresión h_i = (3^i - 1)/2 y se tomará el mayor elemento como el 
# número de la serie que se encuentra 2 posiciones antes del más cercano al largo de la lista
def Shellsort2(a):
    n=len(a)
    K=aux2(n)
    D=[]
    while K>0:
        D.append((3**K-1)//2)
        K-=1
    for d in D:
        pasada_Shellsort(a,d)

# Algoritmo auxiliar que encuentra el k indicado
def aux2(n):
    if n==0:
        return 1
    else:
        for i in range(0,n):
            if (3**i -1)//2>=n:
                K=i
                break
        if abs((3**K - 1)//2 - n) < abs((3**(K-1)//2 - 1) - n):
            return K-2
        else:
            K=K-1
            return K-2

In [150]:
# Verificación del algoritmo shellsort con una nueva secuencia de d_k
A=np.random.random(1000)
Shellsort2(A)
verifica_ordenado(A)

Arreglo ordenado OK.


La información utilizada tiene como referencia a:  
Bustos, B. Estudio y optimización del algoritmo de
ordenamiento Shellsort. https://users.dcc.uchile.cl/~bebustos/files/Bus99.pdf